In [26]:
#reloads source files to get most recent edits
import importlib
importlib.reload(getImageArrays)

<module 'getImageArrays' from '../Functions/getImageArrays.py'>

In [80]:
import sys
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
sys.path.append("../Functions/")
# sys.path.remove("../Functions/")
from getImageArrays import getExamples
import getImageArrays
from importlib import reload

In [62]:
image_path = "../Data/Logistic_Regression_Data/"
num_images = 100
examples_per_batch = 16

In [63]:
train_x, train_y, test_x, test_y = getExamples(side_length=100, image_path=image_path , test_ratio=.2, max_num_images=num_images)
train_x, train_y, test_x, test_y  = train_x.T, train_y.T, test_x.T, test_y.T
assert((len(train_x) + len(test_x)) == num_images), "Not sufficeint number of images" 

ValueError: could not broadcast input array from shape (100,100,3) into shape (100,100)

In [ ]:
def createBatches(x, examples_per_batch):
    final = []
    for i in range(0, len(x), examples_per_batch):
        start = i
        end = start + examples_per_batch
        #creates exclusively
        if end >= len(x):
            return np.array(final)
        final += [x[start:end]]
    return np.array(final)

In [ ]:
train_x, train_y = createBatches(train_x, examples_per_batch), createBatches(train_y, examples_per_batch)
test_x, test_y  = createBatches(test_x, examples_per_batch), createBatches(test_y, examples_per_batch)

In [ ]:
print("train_x", train_x.shape)
print("train_y", train_y.shape)
print("test_x", test_x.shape)
print("test_y", test_y.shape)

In [64]:
np.random.seed(1)
train_x = np.random.randn(1080, 64, 64, 3)
train_y = np.random.randn(1080, 6)
test_x = np.random.randn(120, 64, 64, 3)
test_y = np.random.randn(120, 6)

In [65]:
def create_placeholders(width, height, channels, num_output_classes):
    placeholder_x = tf.placeholder(tf.float32, shape=(None, width, height, channels))
    placeholder_y = tf.placeholder(tf.float32, shape=(None, num_output_classes))
    return placeholder_x, placeholder_y

In [66]:
placeholder_x, placeholder_y = create_placeholders(64, 64, 3, 6)
print(placeholder_x)
print(placeholder_y)

Tensor("Placeholder_2:0", shape=(?, 64, 64, 3), dtype=float32)
Tensor("Placeholder_3:0", shape=(?, 6), dtype=float32)


In [99]:
def initialize_params():
    tf.set_random_seed(1)
    #shape = (window_height, window_width, prev_channels, new_channels)
    W1 = tf.get_variable("W1", shape=[4, 4, 3, 8], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W2 = tf.get_variable("W2", shape=[2, 2, 8, 16], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    #tf takes care of biases
    return {"W1": W1,"W2":W2}

In [100]:
tf.reset_default_graph()
with tf.Session() as sess:
    params = initialize_params()
    initialize = tf.global_variables_initializer()
    sess.run(initialize)
    print(params["W1"].eval()[1, 1, 1])
    print(params["W2"].eval()[1, 1, 1])

[ 0.00131723  0.14176141 -0.04434952  0.09197326  0.14984085 -0.03514394
 -0.06847463  0.05245192]
[-0.08566415  0.17750949  0.11974221  0.16773748 -0.0830943  -0.08058
 -0.00577033 -0.14643836  0.24162132 -0.05857408 -0.19055021  0.1345228
 -0.22779644 -0.1601823  -0.16117483 -0.10286498]


[ 0.00131723  0.14176141 -0.04434952  0.09197326  0.14984085 -0.03514394
 -0.06847463  0.05245192]
[-0.08566415  0.17750949  0.11974221  0.16773748 -0.0830943  -0.08058
 -0.00577033 -0.14643836  0.24162132 -0.05857408 -0.19055021  0.1345228
 -0.22779644 -0.1601823  -0.16117483 -0.10286498]

In [97]:
def foward_propagate(x, params):
    #tf useses params deduce meta data about architecture
    W1 = params["W1"]
    W2 = params["W2"]
    
    # Layer 1: Convolution -> Relu -> Max Pool
    z1 = tf.nn.conv2d(x, W1, strides=[1,1,1,1], padding="SAME")
    a1 = tf.nn.relu(z1)
    p1 = tf.nn.max_pool(a1, ksize=[1, 8, 8, 1], strides=[1, 8, 8, 1], padding="SAME")
    
    # Layer 2: Convolution -> Relu -> Max Pool
    z2 = tf.nn.conv2d(p1, W2, strides=[1,1,1,1], padding="SAME")
    a2 = tf.nn.relu(z2)
    p2 = tf.nn.max_pool(a2, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding="SAME")
    
    # 
    flatten_cnn_output = tf.contrib.layers.flatten(p2)
    preds = tf.contrib.layers.fully_connected(flatten_cnn_output, 6, activation_fn=None)
    
    return preds    

In [102]:
# np.random.seed(17)
tf.reset_default_graph()
with tf.Session() as sess:
    np.random.seed(1)
    placeholder_x, placeholder_y = create_placeholders(64, 64, 3, 6)
    params = initialize_params()
    preds = foward_propagate(placeholder_x, params)
    initialize = tf.global_variables_initializer()
    sess.run(initialize)
    test = sess.run(preds, {placeholder_x: np.random.randn(2, 64, 64, 3), placeholder_y: np.random.randn(2, 6)})
print(test)

[[ 1.44169843 -0.24909666  5.45049906 -0.26189619 -0.20669907  1.36546707]
 [ 1.40708458 -0.02573211  5.08928013 -0.48669922 -0.40940708  1.26248586]]


[[ 1.44169843 -0.24909666  5.45049906 -0.26189619 -0.20669907  1.36546707]
 [ 1.40708458 -0.02573211  5.08928013 -0.48669922 -0.40940708  1.26248586]]

In [92]:
def cost_op(preds, y):
    costs = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y)
    #gets average
    avg_cost = tf.reduce_mean(costs)
    return avg_cost

In [106]:
# np.random.seed(17)
tf.reset_default_graph()
with tf.Session() as sess:
    np.random.seed(1)
    placeholder_x, placeholder_y = create_placeholders(64, 64, 3, 6)
    params = initialize_params()
    preds = foward_propagate(placeholder_x, params)
    print(preds.shape)
    avg_cost = cost_op(preds, placeholder_y)
    
    initialize = tf.global_variables_initializer()
    sess.run(initialize)
    
    test = sess.run(avg_cost, {placeholder_x: np.random.randn(4, 64, 64, 3), placeholder_y: np.random.randn(4, 6)})
print(test)

(?, 6)
13.0227


In [88]:
def model(trainX, trainY, testX, testY, epoch, lr, batch_size):
    ops.reset_default_graph()
    tf.set_random_seed(17)
    num_train_examples , width, height, num_channels = trainX.shape
    num_test_examples , _, _, _ = testX.shape
    output_num_classes = 6
    costs = []
    
    # Conv -> ....
    placeholder_x, placeholder_y = create_placeholders(width, height, num_channels, output_num_classes)
    params = initialize_params()
    preds = foward_propagate(placeholder_x, params)
    avg_cost = cost_op(preds, placeholder_y)
    optimizer = tf.train.AdamOptimizer(lr).minimize(avg_cost)
    
    initialize = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(initialize)
        
        for i in range(epoch):
    #         mini_batch_cost = 0
    #         num_mini_batches = int(num_train_examples/batch_size)

    #         mini_batch_cost = sess.run(avg_cost, {placeholder_x: np.random.randn(4, 64, 64, 3), placeholder_y: np.random.randn(4, 6)})
    #         costs += mini_batch_cost
            cur_cost = sess.run([optimizer, avg_cost], {placeholder_x: trainX, placeholder_y: trainY})
            costs += cur_cost
            print(cur_cost)
    plt.plot(np.squeeze(cur_cost))
    plt.ylabel("cost")
    plt.xlabel("iters")
#     plt.titl
    plt.show()
    
    predict = tf.argmax(preds, 1)
    true_label = tf.argmax(placeholder_y, 1)
    correct = tf.equal(predict, true_label)
    
    accuracy = tf.reduce_mean(tf.cast(correct, "float"))
    train_acc = accuracy.eval({placeholder_x: trainX, placeholder_y: trainY})
    test_acc = accuracy.eval({placeholder_x: testX, placeholder_y: testY})
    print("accuracy", accuracy)
    print("train_acc", train_acc)
    print("test_acc", test_acc)
    
    return train_acc, test_acc, params

In [89]:
print("train_x", train_x.shape)
print("train_y", train_y.shape)
print("test_x", test_x.shape)
print("test_y", test_y.shape)

train_x (1080, 64, 64, 3)
train_y (1080, 6)
test_x (120, 64, 64, 3)
test_y (120, 6)


In [93]:
model(train_x, train_y, test_x, test_y, 200, .009, 4)

[None, -0.0055805207]
[None, -0.035068963]
[None, -0.04212977]
[None, -0.029380748]
[None, -0.0098932339]


KeyboardInterrupt: 